In [9]:
import numpy as np
import pandas as pd
from case_studies import *
import time


In [10]:
#Save case study functions, their derivatives and hessians in lists
fs = [f1, f2, f3, f4, f5]
dfs = [df1, df2, df3, df4, df5]
Hfs = [Hf1, Hf2, Hf3, Hf4, Hf5]
fnames = ["f1", "f2", "f3", "f4", "f5"]

In [11]:
def backtracking_line_search(f, df, x, pk, alpha_init, c1, rho):
    """
    Performs backtracking line search to find a suitable step size.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        x: Current point.
        p: Search direction.
        alpha_init: Initial step size.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
    
    Returns:
        alpha: Suitable step size.
    """
    alpha = alpha_init
    while f(x + alpha * pk) > f(x) + c1 * alpha * np.dot(df(x), pk):
        alpha *= rho
    return alpha

## Steepest Descent

In [12]:
#Steepest Descent

def steepest_descent(f, df, x0, c1=0.01, rho=0.5, tol=1e-6, max_iters=1000):
    """
    Implements the Steepest Descent Algorithm with Backtracking Line Search.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        x0: Initial point.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
        tol: Tolerance for stopping condition.
        max_iters: Maximum number of iterations.
    
    Returns:
        x: Estimated minimum.
        xs: List of iterates for analysis.
    """
    x=x0
    beta = 1.0
    xs = [x0]
    
    for _ in range(max_iters):
        pk = -df(x)
        if np.linalg.norm(pk) < tol:
            break
        
        alpha = backtracking_line_search(f, df, x, pk, beta, c1, rho)
        x = x + alpha * pk
        beta = alpha / rho
        xs.append(x)
    
    return x, xs


## Newton's Algorithm

In [13]:
#Newton's Algorithm

def newtons_method(f, grad_f, hessian_f, x0, c1=1e-4, rho=0.9, tol=1e-6, max_iters=1000):
    """
    Implements Newton's Method with a modified Hessian when necessary.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        hessian_f: Hessian (second derivative) of f.
        x0: Initial point.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
        tol: Tolerance for stopping condition.
        max_iters: Maximum number of iterations.
    
    Returns:
        x: Estimated minimum.
        xs: List of iterates for analysis.
    """
    x = x0
    xs = [x0]
    
    for _ in range(max_iters):
        grad = grad_f(x)
        hessian = hessian_f(x)
        
        if np.linalg.norm(grad) < tol:
            break 
        
        if np.all(np.linalg.eigvals(hessian) > 0):
            p = -np.linalg.solve(hessian, grad)
        else:
            eigvals, eigvecs = np.linalg.eigh(hessian)
            H = sum((1 / abs(eigval)) * np.outer(eigvec, eigvec) for eigval, eigvec in zip(eigvals, eigvecs))
            p = -H @ grad
        
        alpha = backtracking_line_search(f, grad_f, x, p, 1.0, c1, rho)
        x = x + alpha * p
        xs.append(x)
    
    return x, xs

In [14]:
def benchmark(f, df, optimizer, x0, x_opt, Hf):
    start_time = time.time()

    if Hf is not None:
        x_final, xs = optimizer(f, df, Hf, x0)
    else:
        x_final, xs = optimizer(f, df, x0)

    end_time = time.time()

    num_iterations = len(xs)
    final_solution_point = x_final
    dist_to_optimum = np.linalg.norm(x_final-x_opt)
    final_fun_value = f(xs[-1])
    duration = end_time-start_time
    
    grad_norms = [np.linalg.norm(df(x)) for x in xs]

    return (num_iterations, duration, final_fun_value, final_solution_point, dist_to_optimum, grad_norms)


In [ ]:
x0 = np.random.randn(2) 
benchmark_results = []

for f, df, Hf, fname in zip(fs, dfs, Hfs, fnames):
    x_optimal = x_opt(fname, len(x0))

    sd_result = benchmark(f, df, steepest_descent, x0, x_optimal, Hf=None)
    benchmark_results.append((fname, "Steepest Descent") + sd_result)

    nm_result = benchmark(f, df, newtons_method, x0, x_optimal, Hf)
    benchmark_results.append((fname, "Newton's Method") + nm_result)

# Convert to DataFrame
columns = ["Function", "Optimizer", "Iterations", "Time", "Final Function Value", "Final Solution Point", "Distance to Optimum", "Gradient Norms"]
df_results = pd.DataFrame(benchmark_results, columns=columns)


df_results

,Function,Optimizer,Iterations,Time,Final Function Value,Final Solution Point,Distance to Optimum,Gradient Norms
0,f1,Steepest Descent,1001,0.086736,2.987738e-03,"[-0.05451124476299609, 0.00012752489221139093]",5.451139e-02,"[2043.4082715409347, 1947.623524120612, 1856.3..."
1,f1,Newton's Method,2,0.000000,0.000000e+00,"[0.0, 0.0]",0.000000e+00,"[2043.4082715409347, 0.0]"
2,f2,Steepest Descent,1001,0.007020,8.619215e-03,"[0.9071850100816599, 0.822770342482661]",2.000624e-01,"[219.00063243410915, 238.11897426372138, 80.39..."
3,f2,Newton's Method,20,0.013915,7.640680e-17,"[0.9999999995361827, 0.9999999981994869]",1.859294e-09,"[219.00063243410915, 221.02210862483815, 3.966..."
4,f3,Steepest Descent,1001,0.086283,3.012437e+00,"[-0.043880526687094516, 9.049897878212562e-05]",4.388062e-02,"[1.9571951041307112, 2.1375011361928764, 14.86..."
5,f3,Newton's Method,7,0.000000,2.526452e-25,"[-9.681752023239163e-16, 1.5597164436450445e-16]",9.806581e-16,"[1.9571951041307112, 4.603086132057634, 739.76..."
6,f4,Steepest Descent,12,0.000000,1.220319e-05,"[0.001909382756704566, 0.001909382756704566]",7.307522e-08,"[82.5059966641859, 4.7350641785137455, 4.14318..."
7,f4,Newton's Method,8,0.000000,1.220319e-05,"[0.001909331076337135, 0.001909331076337135]",1.185752e-11,"[82.5059966641859, 0.09704555620338283, 0.0301..."
8,f5,Steepest Descent,14,0.000962,1.278106e-09,"[0.0, -0.005979181392456661]",5.979181e-03,"[4.345144033172706, 0.4124035997966812, 0.0003..."
9,f5,Newton's Method,14,0.000000,7.595476e-10,"[0.0, 0.005249751523622587]",5.249752e-03,"[4.345144033172706, 1.2640421681122211, 0.3745..."


In [18]:
grad_norms = df_results["Gradient Norms"]
grad_norms[0]

[np.float64(2043.4082715409347),
 np.float64(1947.623524120612),
 np.float64(1856.3286874258),
 np.float64(1769.3132969223577),
 np.float64(1686.3767536025853),
 np.float64(1607.327861538735),
 np.float64(1531.984387113705),
 np.float64(1460.1726389127955),
 np.float64(1391.727067308045),
 np.float64(1326.4898828120517),
 np.float64(1264.3106923214639),
 np.float64(1205.046152411559),
 np.float64(1148.5596388826457),
 np.float64(1094.7209317964794),
 np.float64(1043.405915276601),
 np.float64(994.4962913805398),
 np.float64(947.8793073842593),
 np.float64(903.4474958501517),
 np.float64(861.098426879348),
 np.float64(820.7344719772075),
 np.float64(782.2625789876163),
 np.float64(745.5940575772477),
 np.float64(710.6443747752527),
 np.float64(677.3329600970335),
 np.float64(645.5830198028472),
 np.float64(615.3213598630429),
 np.float64(586.4782172218114),
 np.float64(558.9870989704527),
 np.float64(532.7846290594051),
 np.float64(507.8104021956552),
 np.float64(484.00684458871496),
 n